<a href="https://colab.research.google.com/github/Muhaiminul-Kabir/My-codes/blob/main/Driver_image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GPU info

In [ ]:
!mkdir crop

In [ ]:
!nvidia-smi

# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#OS

In [ ]:
import os
HOME = os.getcwd()
HOME

#Installing Ultralytics

In [ ]:
!pip install ultralytics

In [ ]:
from IPython import display
import cv2 as cv
import matplotlib.pyplot as plt
import math
import numpy as np
from PIL import Image
from ultralytics import YOLO
from IPython.display import display, Image,clear_output

#Video to frames

#Cropping person/driver

In [ ]:
def crop_driver(path,newp):
  model = YOLO(f'{HOME}/yolov8x-seg.pt')
  results = model.predict(source = path, save=True, save_txt=True,classes=[2,7])
  result = results[0]
  try:
    seg = result.masks[0]
  except:
    print("Aborting process...")
    return
  vehicle_seg_pts = []
  seg_pts = []

  for i in result.masks:
    print(len(i.xy[0]))
    seg_pts.append(len(i.xy[0]))

  maxw = max(seg_pts)


  for i in result.masks:

    if len(i.xy[0]) == maxw:
      vehicle_seg_pts.append(i.xy[0])

  print(f" no of vehicle segmentation coordinates -> {vehicle_seg_pts}")

  npath = crop_seg(path,vehicle_seg_pts)

  if npath :
    detect_driver_crop(npath,model)
  else:
    return




#Croping a segment

In [ ]:
def crop_seg(path,vehicle_seg_pts):
  img  = cv.imread(path)

  pts = np.array(vehicle_seg_pts[0])

  rect = cv.boundingRect(pts)
  x,y,w,h = rect
  croped = img[y:y+h, x:x+w].copy()

  pts = pts - pts.min(axis=0)

  np.ceil(pts)
  print(f"segments co ordinates -> {pts}")
  mask = np.zeros(croped.shape[:2], np.uint8)
  cv.drawContours(mask, [pts.astype(int)], -1, (255, 255, 255), -1, cv.LINE_AA)

  dst = cv.bitwise_and(croped, croped, mask=mask)

  bg = np.ones_like(croped, np.uint8)*255
  cv.bitwise_not(bg,bg, mask=mask)
  dst2 = bg+ dst


  cv.imwrite("/content/dst2.png", dst2)
  return "/content/dst2.png"

#Driver detection and croping

In [ ]:
def detect_driver_crop(npath,model):
  results1 = model.predict(source = npath, save=True, save_txt=True,classes=[0])

  result2 = results1[0]


  person_cords = []
  person_a = []

  for i in result2.boxes:
    print(i.xyxy.tolist())
    _,_,w,h = i.xywh[0].tolist()
    _,_,_,y2 = i.xyxy[0].tolist()
    person_a.append(w*h)
  print(f'jdks {person_a}')
  if len(person_a) < 1:
    print('No driver in the image...')
    return
  maxw = max(person_a)


  for i in result2.boxes:
    print(i.xyxy.tolist())
    _,_,w,h = i.xywh[0].tolist()
    if w*h == maxw:
      person_cords.append(i.xyxy[0].tolist())

  print(person_cords)

  x1,y1,x2,y2 = person_cords[0]

  x1 = math.floor(x1)
  y1 = math.floor(y1)
  x2 = math.ceil(x2)
  y2 = math.ceil(y2)


  img  = cv.imread(npath)
  print(y2)
  if y2 < 2000:
    y2 += 300
  if x2 < 2000:
    x2 += 300
  print(img.shape)
  img = img[y1:y2,x1:x2]
  print('Driver detected...')
  cv.imwrite(f"{newp}", img)
  plt.imshow(img)



#Path set and start process

In [ ]:
dir = '/content/drive/MyDrive/DATASET/imbalanced dataset/'
lvl = ['one hand out','one hand driving','looking outside','using phone', 'safe driving ', 'hair and makeup']

In [ ]:
cnt = 0
for l in lvl:
    fld = os.path.join(dir,l)

    print(fld)
    yes = 0
    for img in os.listdir(fld):
        path = os.path.join(fld,img)
        x = path.split('/')
        label = x[6]
        name = x[7]
        nd = f'/content/crop/{label}'

        newp = f'{nd}/{name}'
        if yes == 0 :
          os.mkdir(nd)
          yes = 1


        crop_driver(path,newp)
        cnt+=1
        print(f'Data no -> {cnt}')




#Download folders

In [ ]:
from google.colab import files

files.download('/content/crop/')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Collaborator

Md. Muhaiminul Kabir<br>
[Github](https://github.com/Muhaiminul-Kabir)